In [ ]:
!pip install pytesseract evaluate tqdm transformers datasets rouge-score accelerate nltk tensorboard jupyter-black py7zr --upgrade
!apt-get install git --yes
!apt-get install git-lfs --yes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:

In [ ]:
import json
import pandas
import jupyter_black
from datetime import timedelta
from datasets import Dataset

In [ ]:
with open("data.json", "r") as f:
    data = json.load(f)

# create a dataframe
df = pandas.DataFrame(data)

data = Dataset.from_pandas(df)

data

Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score'],
    num_rows: 6
})

In [ ]:
data[1]['task']

'Rate the answer to the question.'

In [ ]:
from transformers import BertTokenizer, BertModel

MODEL_NAME = "C++ teacher AI"  # the name of your model
MODEL_ID = "roberta-base"  # the id of the base model we will train (can be small, base, large, xl, etc.) (the bigger - the more GPU memory you need)
NUM_TRAIN_EPOCHS = 4  # number of epochs to train

#tokenizer = BertTokenizer.from_pretrained(MODEL_ID)
#model = BertModel.from_pretrained(MODEL_ID)

In [ ]:
def x_shot_inference(example_indeces, inference_index):
  text = ""
  line_break = '\n'
  for i in example_indeces:
    prompt = f"""
          {data[i]['task']}

          Question:
          {data[i]['question']}

          Maximum score:
          {data[i]['maximum_score']}

          Scoring guide:
          {line_break.join(str(str(z['point']) + " points - " + z['criteria']) for z in data[i]['scoring_guide'])}

          Answer:
          {data[i]['answer']}

          Score:
          {data[i]['score']}



        """
    text += prompt

  prompt = f"""
          {data[inference_index]['task']}

          Question:
          {data[inference_index]['question']}

          Maximum score:
          {data[inference_index]['maximum_score']}

          Scoring guide:
          {line_break.join(str(str(z['point']) + " points - " + z['criteria']) for z in data[inference_index]['scoring_guide'])}

          Answer:
          {data[inference_index]['answer']}

          Score:
          <mask>
      """

  text += prompt

  return text

In [ ]:
examples = [0]
to_infere = 3

text = x_shot_inference(examples, to_infere)

print(text)


          Rate the answer to the question.

          Question:
          Explain the difference between malloc() and new in C++.

          Maximum score:
          6

          Scoring guide:
          2 points - malloc() is a C function used for dynamic memory allocation, while new is a C++ operator that also allocates memory dynamically.
2 points - Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete.
2 points - new is type-safe, whereas malloc() returns a void pointer, requiring explicit type casting.

          Answer:
          malloc() is a C function used for dynamic memory allocation, while new is a C++ operator that also allocates memory dynamically. new not only allocates memory but also calls the constructor for an object. Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete. Additionally, new i

In [ ]:
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

print(output)

NameError: ignored

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model=MODEL_ID)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.3122478723526001,
  'token': 305,
  'token_str': '6',
  'sequence': 'Rate the answer to the question. Question: Explain the difference between malloc() and new in C++. Maximum score: 6 Scoring guide: 2 points - malloc() is a C function used for dynamic memory allocation, while new is a C++ operator that also allocates memory dynamically. 2 points - Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete. 2 points - new is type-safe, whereas malloc() returns a void pointer, requiring explicit type casting. Answer: malloc() is a C function used for dynamic memory allocation, while new is a C++ operator that also allocates memory dynamically. Additionally, new is type-safe, whereas malloc() returns a void pointer, requiring explicit type casting. Score: 4 Rate the answer to the question. Question: Explain the difference between malloc() and new in C++. Maximum score: 6 Scoring guide: 2 poin

In [ ]:
unmasker(text)

[{'score': 0.5225058197975159,
  'token': 305,
  'token_str': '6',
  'sequence': 'Rate the answer to the question. Question: Explain the difference between malloc() and new in C++. Maximum score: 6 Scoring guide: 2 points - malloc() is a C function used for dynamic memory allocation, while new is a C++ operator that also allocates memory dynamically. 2 points - Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete. 2 points - new is type-safe, whereas malloc() returns a void pointer, requiring explicit type casting. Answer: malloc() is a C function used for dynamic memory allocation, while new is a C++ operator that also allocates memory dynamically. new not only allocates memory but also calls the constructor for an object. Memory allocated with malloc() must be explicitly deallocated with free(), whereas memory allocated with new should be deallocated with delete. Additionally, new is type-safe, 